In [1]:
!nvidia-smi

Mon Jun 10 05:17:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [3]:
device = 'cuda'

In [5]:
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Linear(in_featur

# Generation 1

In [6]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Generation 2 (Streaming)

In [8]:
from transformers import TextStreamer

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
    streamer = TextStreamer(tokenizer)
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant
A large language model is a type of artificial intelligence that can generate human-like text by analyzing vast amounts of text data and using machine learning algorithms. These models are trained on large datasets, which means they have access to vast amounts of information about the world. The goal of a large language model is to generate human-like text based on the patterns and structures in this data.

Large language models are used for a variety of applications, including natural language processing, speech recognition, question answering, and chatbots. They can also be used as a tool for generating creative writing or poetry. However, their use raises ethical concerns, such as privacy, bias, and transparency, as well as concerns about their ability to reason logically and make decisions.<|im_end|>


# Generation 3 (Prompt)

In [9]:
from transformers import TextStreamer

prompt = "거대 언어 모델에 대해서 짧게 설명해주세요."
messages = [
    {"role": "system", "content": "당신은 한국어 AI 어시스턴트입니다."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512,
    streamer = TextStreamer(tokenizer)
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

<|im_start|>system
당신은 한국어 AI 어시스턴트입니다.<|im_end|>
<|im_start|>user
거대 언어 모델에 대해서 짧게 설명해주세요.<|im_end|>
<|im_start|>assistant
거대 언어 모델은 빠르고 정확한 문장을 생성하는 모델을 말합니다. 이 모델들은 대규모 데이터를 사용하여 학습하고, 이를 통해 다양한 언어와 문장의 이해를 돕습니다. 이는 텍스트 분석, 자연어 처리, 인공지능 기술 등에서 중요한 역할을 합니다. 거대 언어 모델은 단일 문장을 생성하거나 여러 문장을 생성하는 능력이 있습니다.<|im_end|>


In [10]:
def generation(prompt):

  messages = [
      {"role": "system", "content": "당신은 한국어 AI 어시스턴트입니다."},
      {"role": "user", "content": prompt}
  ]
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(device)

  generated_ids = model.generate(
      model_inputs.input_ids,
      max_new_tokens=512,
      streamer = TextStreamer(tokenizer)
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

In [14]:
generation('서울대학교에 대해서 알려줘')

<|im_start|>system
당신은 한국어 AI 어시스턴트입니다.<|im_end|>
<|im_start|>user
서울대학교에 대해서 알려줘<|im_end|>
<|im_start|>assistant
죄송합니다, 서울대학교의 특정 정보나 정보를 제공하는 것은 제한적일 수 있습니다. 제가 필요한 정보를 찾지 못했습니다. 다른 질문이 있으시면 도와드릴 수 있을 것 같습니다.<|im_end|>


'죄송합니다, 서울대학교의 특정 정보나 정보를 제공하는 것은 제한적일 수 있습니다. 제가 필요한 정보를 찾지 못했습니다. 다른 질문이 있으시면 도와드릴 수 있을 것 같습니다.'

In [15]:
generation('BERT와 GPT의 차이점')

<|im_start|>system
당신은 한국어 AI 어시스턴트입니다.<|im_end|>
<|im_start|>user
BERT와 GPT의 차이점<|im_end|>
<|im_start|>assistant
BERT (Bidirectional Encoder Representations from Transformers)와 GPT (Generative Pre-trained Transformer)는 두 가지 다른 인공지능 기술을 사용하는 모델입니다.

1. **BERT**: BERT은 2018년에 발표된 학습 알고리즘으로, 대체적으로 인공지능 분류 모델로 유명합니다. 이 모델은 단순히 문장에서의 정보를 추출하고, 이를 통해 훈련된 인공지능을 사용하여 문장에서의 의미를 추적할 수 있게 해줍니다. 이 모델은 매우 빠른 성능과 정확성을 가지고 있습니다.

2. **GPT**: GPT는 2017년에 발표된 학습 알고리즘이며, 인공지능 언어 모델로 유명합니다. GPT-3는 GPT-2의 최신 버전이며, 이는 대략적인 문장을 설명하거나 질문에 답하기 위해 사용됩니다. GPT-3은 더 높은 정확도를 보여주고, 이는 학습 시간과 성능을 줄이고, 많은 사람들이 사용하기 쉬운 언어 모델로 성장했습니다.

두 모델 모두 대상 시스템을 사용하며, BERT은 문장에서의 정보를 추적하고, GPT는 대규모 문장을 설명하거나 질문에 답하는 것입니다.<|im_end|>


'BERT (Bidirectional Encoder Representations from Transformers)와 GPT (Generative Pre-trained Transformer)는 두 가지 다른 인공지능 기술을 사용하는 모델입니다.\n\n1. **BERT**: BERT은 2018년에 발표된 학습 알고리즘으로, 대체적으로 인공지능 분류 모델로 유명합니다. 이 모델은 단순히 문장에서의 정보를 추출하고, 이를 통해 훈련된 인공지능을 사용하여 문장에서의 의미를 추적할 수 있게 해줍니다. 이 모델은 매우 빠른 성능과 정확성을 가지고 있습니다.\n\n2. **GPT**: GPT는 2017년에 발표된 학습 알고리즘이며, 인공지능 언어 모델로 유명합니다. GPT-3는 GPT-2의 최신 버전이며, 이는 대략적인 문장을 설명하거나 질문에 답하기 위해 사용됩니다. GPT-3은 더 높은 정확도를 보여주고, 이는 학습 시간과 성능을 줄이고, 많은 사람들이 사용하기 쉬운 언어 모델로 성장했습니다.\n\n두 모델 모두 대상 시스템을 사용하며, BERT은 문장에서의 정보를 추적하고, GPT는 대규모 문장을 설명하거나 질문에 답하는 것입니다.'

In [16]:
generation('Can you speak English?')

<|im_start|>system
당신은 한국어 AI 어시스턴트입니다.<|im_end|>
<|im_start|>user
Can you speak English?<|im_end|>
<|im_start|>assistant
Yes, I can understand and respond in English. How may I assist you today?<|im_end|>


'Yes, I can understand and respond in English. How may I assist you today?'

In [17]:
generation('자연어처리와 관련된 보고서 작성을 해줘')

<|im_start|>system
당신은 한국어 AI 어시스턴트입니다.<|im_end|>
<|im_start|>user
자연어처리와 관련된 보고서 작성을 해줘<|im_end|>
<|im_start|>assistant
죄송합니다, 저는 자연어 처리 (NLP)에 대한 이해를 가지고 있지 않습니다. 하지만 제가 도움을 드릴 수 있는 다른 질문이 있으시다면 말씀해주세요.<|im_end|>


'죄송합니다, 저는 자연어 처리 (NLP)에 대한 이해를 가지고 있지 않습니다. 하지만 제가 도움을 드릴 수 있는 다른 질문이 있으시다면 말씀해주세요.'

In [20]:
generation('맥북의 개발자 세종대왕에 대해 알려줘')

<|im_start|>system
당신은 한국어 AI 어시스턴트입니다.<|im_end|>
<|im_start|>user
맥북의 개발자 세종대왕에 대해 알려줘<|im_end|>
<|im_start|>assistant
죄송합니다, "맥북"이라는 제품이 직접 개발자인 세종대왕을 제시하지 못했습니다. 그러나 그는 '삼성전자'에서 일하고 있습니다.<|im_end|>


'죄송합니다, "맥북"이라는 제품이 직접 개발자인 세종대왕을 제시하지 못했습니다. 그러나 그는 \'삼성전자\'에서 일하고 있습니다.'

In [ ]:
generation('맥북의 개발자 세종대왕에 대해 알려줘')